Pobieranie wiadomości z pcworld.pl Zapis do pliku ./data/pcworld_{timestamp}.txt

In [1]:
# ! pip install bs4
# ! pip install requests
# ! pip install tqdm
# ! pip install pandas
# ! pip install pyarrow
# ! pip install qgrid
# ! pip install blaze
# ! pip install ipynbname

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, trange
from datetime import datetime, timezone
from common.webarticle import WebArticle
from common.pl_helper import PolishLocaleHelper
import pandas as pd
import logging
import ipynbname

In [3]:
# settings:
portal_name = 'pcworld'
website = 'https://www.pcworld.pl/news/'
website_short = 'https://www.pcworld.pl'
pagination_attributes = 'archiwum-'
no_articles_to_scrap = 300
data_dir = '../data'
logging.basicConfig(
    filename=f'../logs/{ipynbname.name()}.ipynb.log',
    encoding='utf-8',
    level=logging.DEBUG
)
ignored_paragraph_starts = []

Sprawdźmy plik robots.txt, w czasie pisania tego kodu pozwalał na pobieranie plików z portalu pcworld z kategorii news z drobnymi wyjątkami (zakaz dotyczył m.in. pdf-ów i ajaxa)

In [4]:
resp = requests.get('https://www.pcworld.pl/robots.txt')
print(resp.text)

User-Agent: *
Allow: / 
Disallow: /sonda/sonda_news.asp 
Disallow: /sonda/ajax/ 
Disallow: /stats/
Disallow: /block/
Disallow: /ajax/
Disallow: /auth/
Disallow: /auth/block/
Disallow: /comment/block/
Disallow: /download/block/
Disallow: /duel/block/
Disallow: /news/block/
Disallow: /news/ajax/
Disallow: /news/pdf/
Disallow: /poll/block/
Disallow: /quiz/block/
Disallow: /gallery/block/
Disallow: /default/block/


Sitemap: https://www.pcworld.pl/sitemap/sitemap_news.xml
Sitemap: https://www.pcworld.pl/sitemap/sitemap_galeria.xml
Sitemap: https://www.pcworld.pl/sitemap/sitemap_testy.xml
Sitemap: https://www.pcworld.pl/sitemap/sitemap_rankingi.xml
Sitemap: https://www.pcworld.pl/sitemap/sitemap_ftp.xml
Sitemap: https://www.pcworld.pl/sitemap/sitemap_testy_produkty.xml
Sitemap: https://www.pcworld.pl/sitemap/sitemap.xml
Sitemap: https://www.pcworld.pl/sitemap/googlenews.xml



In [5]:
def check_if_ignored_paragraph(text, ignored_paragraphs, operation='startswith'):

    for ignored_paragraph in ignored_paragraphs:
        if operation == 'startswith':
            if text.startswith(ignored_paragraph):
                return True
        else:
            logging.debug(f'check_if_ignored: operation {operation} not implemented')
    return False


In [6]:
def scrap_article_text_pcworld(link):
    logging.info('scrap_article_text_pcworld:')
    logging.info(f'article link to scrap: {link}')

    page = requests.get(link)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return '', ''

    soup = BeautifulSoup(page.content, 'html.parser')
    # paragraphs = soup.find('div', class_='storybody').find_all('p')

    text = soup.find('div', class_='storybody').get_text(separator=' ', strip=True)
    lead_text_int = soup.find('span', itemprop='articleBody description text').find('p').get_text(strip=True)

    time_string = soup.find('div', id='container-article-meta').find('li', itemprop='datePublished').get_text(strip=True)
    # 19.01.2022, godz. 14:53
    when_published = datetime.strptime(time_string, "%d.%m.%Y, godz. %H:%M")

    title_int = soup.find('h1', itemprop='headline name').get_text(strip=True)
    author_candidate = soup.find('div', id='container-article-meta').find('li', class_='author-name')
    if author_candidate is None:
        author_candidate = soup.find('div', id='container-article-meta').find('a', class_='newsautor').get_text(strip=True)
    else:
        author_candidate = author_candidate.get_text(strip=True)
    author = author_candidate
    source = None

    return text, when_published, title_int, lead_text_int, author, source

In [7]:
def scrap_articles_pcworld(website):
    logging.info('scrap_articles_pcworld:')
    logging.info(f'page with articles to scrap: {website}')

    result = list()
    page = requests.get(website)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return result

    soup = BeautifulSoup(page.content, 'html.parser')
    page_content = soup.select('div[class*="row row-list-item separator news-"]')

    print(f'articles to scrap: {len(page_content)}')

    for article in page_content:

        logging.info(f"article scrapping started: {article}")
        title = article.find('span', class_='title').get_text().strip()
        lead_text = article.find('span', class_='description d-none d-md-inline').get_text().strip()
        link = website_short + article.find('a', href=True)['href']
        webart = WebArticle(title, lead_text, link, *scrap_article_text_pcworld(link),portal_name)
        logging.info(f'webarticle scrapped: {webart}')
        result.append(webart)

    return result

In [8]:
no_scrapped_articles = 0
page_no=1
scrapped_articles = list()

logging.info(f'no_articles_to_scrap: {no_articles_to_scrap}')

pbar = tqdm(total=no_articles_to_scrap)
while no_scrapped_articles < no_articles_to_scrap:
    logging.info('next page: '+website+pagination_attributes+str(page_no)+'.html')
    new_articles = scrap_articles_pcworld(website+pagination_attributes+str(page_no)+'.html')
    page_size = len(new_articles)
    logging.info(f'new_articles: {page_size}')
    if no_scrapped_articles + page_size > no_articles_to_scrap:
        page_size = no_articles_to_scrap - no_scrapped_articles
    scrapped_articles.extend(new_articles[:page_size])
    logging.info(f'scrapped new articles: {page_size}')
    no_scrapped_articles += page_size
    pbar.update(page_size)
    logging.info(f'no_scrapped_articles (total): {no_scrapped_articles}')
    page_no += 1
    if len(scrapped_articles) <= 0:
        logging.debug(f'0 articles scrapped!')
        break
pbar.close()

  0%|          | 0/300 [00:00<?, ?it/s]

articles to scrap: 18


  6%|▌         | 18/300 [00:05<01:22,  3.40it/s]

articles to scrap: 20


 13%|█▎        | 38/300 [00:09<01:06,  3.96it/s]

...

 93%|█████████▎| 278/300 [00:59<00:04,  5.44it/s]

articles to scrap: 20


 99%|█████████▉| 298/300 [01:02<00:00,  5.76it/s]

articles to scrap: 20


100%|██████████| 300/300 [01:07<00:00,  4.48it/s]


In [9]:
for i in trange(len(scrapped_articles)):
    # print(f'{i}: {scrapped_articles[i].lead_text}')
    print(f'{i}: {len(scrapped_articles[i].text)}')

100%|██████████| 300/300 [00:00<00:00, 11470.40it/s]

0: 8162
1: 1334
2: 4766
3: 33208
4: 4326
5: 16997
6: 2320
7: 1773
8: 3367
9: 1848
10: 14483
11: 86108
12: 1627
13: 6519
14: 6515
15: 27433
16: 2911
17: 3281
18: 14320
19: 2029
20: 1891
21: 32133
22: 8484
23: 2399
24: 7133
25: 1255
26: 12416
27: 19002
28: 27961
29: 28458
30: 22981
31: 3023
32: 13264
33: 11119
34: 1586
35: 10784
36: 4510
37: 183039
38: 2276
39: 21942
40: 1669
41: 4820
42: 2969
43: 6508
44: 2159
45: 8893
46: 3185
47: 2476
48: 5250
49: 31991
50: 2790
51: 3557
52: 1895
53: 6083
54: 140396
55: 4881
56: 55197
57: 1445
58: 3394
59: 10928
60: 2425
61: 121625
62: 82655
63: 1721
64: 3937
65: 141030
66: 12971
67: 5277
68: 2078
69: 38449
70: 2385
71: 26154
72: 2509
73: 34268
74: 3228
75: 2002
76: 2904
77: 2334
78: 110696
79: 9450
80: 1984
81: 12828
82: 2133
83: 1918
84: 3907
85: 6765
86: 4158
87: 1910
88: 33313
89: 2326
90: 3249
91: 5486
92: 3793
93: 2393
94: 41502
95: 3707
96: 2093
97: 3955
98: 2500
99: 1590
100: 6987
101: 4287
102: 3735
103: 3319
104: 5447
105: 3162
106: 2803
107

In [10]:
pcworld_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)
pcworld_dataframe

,title,title_int,lead_text,lead_text_int,link,text,when_published,author,source,portal
0,Najlepsze gry do wspólnej zabawy na Nintendo S...,Najlepsze gry do wspólnej zabawy na Nintendo S...,"Sprawdź, naszym zdaniem, najlepsze gry koopera...","Sprawdź, naszym zdaniem, najlepsze gry koopera...",https://www.pcworld.pl/news/Najlepsze-gry-do-w...,Konsola Nintendo Switch idealnie nadaje się do...,2022-01-19 18:36:00,"Damian Kubik,",,pcworld
1,Wyciekły rendery nowego średniaka TCL 30. Wygl...,Wyciekły rendery nowego średniaka TCL 30. Wygl...,Firma TCL weszła na rynek smartfonów w 2020 ro...,Firma TCL weszła na rynek smartfonów w 2020 ro...,https://www.pcworld.pl/news/Wyciekly-rendery-n...,Od czasu premiery serii TCL 10 minęły dwa lata...,2022-01-19 18:02:00,"Magda Pilarczyk,",,pcworld
2,Battlefield 2042 - czy aktualizacja 3.2 napraw...,Battlefield 2042 - czy aktualizacja 3.2 napraw...,Battlefield 2042 ma szansę na zyskanie drugieg...,Battlefield 2042 ma szansę na zyskanie drugieg...,https://www.pcworld.pl/news/Battlefield-2042-c...,Battlefield 2042 zadebiutował na rynku ze spor...,2022-01-19 17:59:00,"Damian Kubik,",,pcworld
3,"Historia aktualizacji systemu Windows 10, wers...","Historia aktualizacji systemu Windows 10, wers...",Aktualizacja systemu Windows 10 o nazwie 20H2 ...,Aktualizacja systemu Windows 10 o nazwie 20H2 ...,https://www.pcworld.pl/news/Historia-aktualiza...,Dwie wersje Windows 10 o nazwie 20H2 i 2004 dz...,2022-01-19 17:10:00,"Magda Pilarczyk,",,pcworld
4,"Historia aktualizacji systemu Windows 10, wers...","Historia aktualizacji systemu Windows 10, wers...",Aktualizacja systemu Windows 10 o nazwie 21H2 ...,Aktualizacja systemu Windows 10 o nazwie 21H2 ...,https://www.pcworld.pl/news/Historia-aktualiza...,Oprogramowanie Windows 10 w wersji 21H2 oficja...,2022-01-19 17:09:00,"Magda Pilarczyk,",,pcworld
...,...,...,...,...,...,...,...,...,...,...
295,Dodatek do Humankind jeszcze w styczniu. Co no...,Dodatek do Humankind jeszcze w styczniu. Co no...,Jeszcze w tym miesiącu oficjalnie na rynku zad...,Jeszcze w tym miesiącu oficjalnie na rynku zad...,https://www.pcworld.pl/news/Dodatek-do-Humanki...,Jeszcze w tym miesiącu Humankind otrzyma pierw...,2022-01-12 10:28:00,"Damian Kubik,",,pcworld
296,Adrenalin - opublikowano nowy sterownik dla ka...,Adrenalin - opublikowano nowy sterownik dla ka...,Jakie zmiany wprowadza dla posiadaczy kart mar...,Jakie zmiany wprowadza dla posiadaczy kart mar...,https://www.pcworld.pl/news/Adrenalin-opubliko...,Nowy sterownik Radeon Adrenalin można pobrać b...,2022-01-12 10:17:00,"Henryk Tur,",,pcworld
297,Monster Hunter Rise - dziś premiera na PC - gd...,Monster Hunter Rise - dziś premiera na PC - gd...,Premiera Monster Hunter Rise na PC już dziś. S...,Premiera Monster Hunter Rise na PC już dziś. S...,https://www.pcworld.pl/news/Monster-Hunter-Ris...,fot. Capcom Monster Hunter Rise PC - data prem...,2022-01-12 10:02:00,"Maciej Persona,",,pcworld
298,Radeon Super Resolution zwiększy wydajność gie...,Radeon Super Resolution zwiększy wydajność gie...,"AMD chwali się, że dzięki nowemu, autorskiemu ...","AMD chwali się, że dzięki nowemu, autorskiemu ...",https://www.pcworld.pl/news/Radeon-Super-Resol...,"AMD nie pokazało za dużo na CES 2022 , ale dzi...",2022-01-12 09:53:00,"Henryk Tur,",,pcworld


In [11]:
timestamp = int((datetime.now(timezone.utc)).timestamp())

In [12]:
pcworld_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
pcworld_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')

Krótki test - dane pierwszego artykułu

In [13]:
print(f'title:\n  {pcworld_dataframe.iloc[0]["title"]}')
print(f'lead_text:\n  {pcworld_dataframe.iloc[0]["lead_text"]}')
print(f'lead_text_int:\n  {pcworld_dataframe.iloc[0]["lead_text_int"]}')
print(f'link:\n  {pcworld_dataframe.iloc[0]["link"]}')
print(f'text:\n  {pcworld_dataframe.iloc[0]["text"]}')
print(f'when_published:\n  {pcworld_dataframe.iloc[0]["when_published"]}')

title:
  Najlepsze gry do wspólnej zabawy na Nintendo Switch
lead_text:
  Sprawdź, naszym zdaniem, najlepsze gry kooperacyjne dostępne na Nintendo Switch.
lead_text_int:
  Sprawdź, naszym zdaniem, najlepsze gry kooperacyjne dostępne na Nintendo Switch.
link:
  https://www.pcworld.pl/news/Najlepsze-gry-do-wspolnej-zabawy-na-Nintendo-Switch,430143.html
text:
  Konsola Nintendo Switch idealnie nadaje się do wspólnej zabawy przy jednym ekranie. Bogata biblioteka gier, która jest stale rozszerzana gwarantuje, że każdy znajdzie tutaj coś dla siebie. Przedstawiliśmy kilka, naszym zdaniem, najlepszych gier na Nintendo Switch, w które można zagrać w trybie lokalnej kooperacji . Enter the Gungeon W Enter the Gungeon wcielamy się w postać bohatera, który wyrusza w podróż, aby odnaleźć broń, która może zmienić przeszłość. Fabuła jest jednak tylko pretekstem do rozpoczęcia rozgrywki. Jest to szalony, wypełniony akcją roguelike , w którym czekają na nas hordy przeciwników. Na drodze spotkamy też prz

In [14]:
print(f'title:\n  {pcworld_dataframe.iloc[99]["title"]}')
print(f'lead_text:\n  {pcworld_dataframe.iloc[99]["lead_text"]}')
print(f'link:\n  {pcworld_dataframe.iloc[99]["link"]}')
print(f'text:\n  {pcworld_dataframe.iloc[99]["text"]}')
print(f'when_published:\n  {pcworld_dataframe.iloc[99]["when_published"]}')

title:
  Moon Knight na mrocznym zwiastunie. Zobacz trailer nowego serialu Marvela
lead_text:
  Marvel opublikował nowy zwiastun nadchodzącego serialu Moon Knight.
link:
  https://www.pcworld.pl/news/Moon-Knight-na-mrocznym-zwiastunie-Zobacz-trailer-nowego-serialu-Marvela,435424.html
text:
  Moon Knight to serial Marvela, który opowie historię Marca Spectora , byłego żołnierza i agenta CIA, który cierpi na dysocjacyjne zaburzenia tożsamości. Każda z nich objawia się jako odrębna jednostka. Co więcej, staje się on kanałem przejścia dla egipskiego bóstwa, które nazywane jest Khonshu . Nowe moce tylko pogłębiają stan, w jakim znalazł się bohater grany przez Oscara Isaaca. Marvel opublikował nowy, mroczny zwiastun serialu Moon Knight . Możecie go zobaczyć poniżej. Moon Knight zadebiutuje już 30 marca. Serial będzie można obejrzeć na platformie Disney +. Jeśli zastanawiasz się, jakie filmy i seriale ze świata Marvela będziemy mogli zobaczyć w 2022 roku, zapraszamy do lektury zestawienia wsz

In [15]:
print(f'title:\n  {pcworld_dataframe.iloc[5]["title"]}')
print(f'lead_text:\n  {pcworld_dataframe.iloc[5]["lead_text"]}')
print(f'link:\n  {pcworld_dataframe.iloc[5]["link"]}')
print(f'text:\n  {pcworld_dataframe.iloc[5]["text"]}')
print(f'when_published:\n  {pcworld_dataframe.iloc[5]["when_published"]}')

title:
  Historia aktualizacji systemu Windows 10, wersja 21H1 [19.01.2022]
lead_text:
  Firma Microsoft udostępniła system Windows 10 w wersji 21H1. Od czasu premiery oprogramowanie wzbogaciło się już o kilka aktualizacji. Co zmieniła każda z nich?
link:
  https://www.pcworld.pl/news/Historia-aktualizacji-systemu-Windows-10-wersja-21H1-19-01-2022,428520.html
text:
  Na dzień dzisiejszy są trzy stale aktualizowane wersje systemu Windows 10: 21H1 , 20H2 oraz 2004 . Wszystkie trzy dzielą wspólny podstawowy system operacyjny oraz identyczny zestaw plików systemowych. Najnowszy za nich, 21H1, dostępny jest od maja tego roku i już został rozszerzony o kilka aktualizacji. W poniższej analizie omówimy wszystkie poprawki udostępnione dla systemu Windows 10 w wersji 21H1 . Co ważne, identyczne zmiany wypisane w poniższym artykule dostępne są także dla oprogramowań w wersji 20H2 oraz 2004, jednakże uwaga jest taka, że kompilacja kierowana dla 2004 nie zawiera poprawek dla wyższych rangą wersji. 